In [17]:
# imports
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv
from matplotlib import pyplot as plt
import seaborn as sns

In [18]:
df_google = pd.read_csv('hotels_in_toronto.csv')

In [22]:
# Using dotenv to retrieve Yelp API key
load_dotenv()
api_key_y_string = 'YELP_API_KEY2' # replace with your Yelp API key variable name
api_key_y = os.getenv(api_key_y_string)

In [21]:
df_google.head()

,Unnamed: 0,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,...,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,plus_code.compound_code,plus_code.global_code,opening_hours.open_now
0,0,OPERATIONAL,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Hilton Toronto,"[{'height': 833, 'html_attributions': ['<a hre...",ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,...,4747,43.649852,-79.385576,43.651296,-79.384380,43.648596,-79.387080,"JJX7+WQ Toronto, Ontario",87M2JJX7+WQ,NaN
1,1,OPERATIONAL,"1 King St W, Toronto, ON M5H 1A1, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,One King West Hotel & Residence,"[{'height': 4931, 'html_attributions': ['<a hr...",ChIJG74DgzLL1IkRu7-_ro1TbdY,4.4,...,4771,43.648873,-79.378072,43.650154,-79.376654,43.647454,-79.379354,"JJXC+GQ Toronto, Ontario",87M2JJXC+GQ,True
2,2,OPERATIONAL,"7095 Woodbine Ave, Markham, ON L3R 1A3, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Courtyard by Marriott Toronto Northeast/Markham,"[{'height': 320, 'html_attributions': ['<a hre...",ChIJVy3BfGXT1IkRjZhm3zfseUE,4.2,...,1438,43.816659,-79.348556,43.818135,-79.347102,43.815435,-79.349801,"RM82+MH Markham, Ontario",87M2RM82+MH,NaN
3,3,OPERATIONAL,"600 Hwy 7, Richmond Hill, ON L4B 1B2, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Best Western Parkway Hotel Toronto North,"[{'height': 668, 'html_attributions': ['<a hre...",ChIJO08zIUsrK4gRH2N9rmvgXAI,3.9,...,945,43.845471,-79.381658,43.846819,-79.380389,43.844120,-79.383089,"RJW9+58 Richmond Hill, Ontario",87M2RJW9+58,NaN
4,4,OPERATIONAL,"1760 Finch Ave E, North York, ON M2J 5G3, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Residence & Conference Centre - Toronto,"[{'height': 427, 'html_attributions': ['<a hre...",ChIJP5KtLRvT1IkRotSzNl83vAI,3.8,...,459,43.796139,-79.346939,43.797524,-79.345362,43.794824,-79.348061,"QMW3+F6 North York, Toronto, ON",87M2QMW3+F6,True


In [ ]:
# Definging the function to find all POI's in Yelp of a specific category within a set distance from any bike stations
def find_POIs_yelp(station_ids,latitudes,longitudes,radius,limit,categories):
    
    #fields = 'fsq_id,name,rating,popularity,price'
    list_POI = []
    
    # Find all POI's within 1000m of each bike station.  This will return one column with station_id and one column containing lists of POI's.
    for stat_ids,lat,long in zip(station_ids,latitudes,longitudes):
        url = f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius={radius}&limit={limit}&categories={categories}'
        headers = {"accept": "application/json","Authorization": 'bearer ' + api_key_y}
        response = requests.get(url, headers=headers).json()['businesses']
        list_POI.append([stat_ids,response])
    
    # convert to dataframe
    df = pd.DataFrame(list_POI,columns=['station_id','businesses'])

    # create 1 row for each match of bike station to POI.  There are duplicates of both bike stations and POI's, but each matchup is unique.
    df = df.explode('businesses').reset_index(drop=True) 
    df = pd.concat([df,pd.json_normalize(df['businesses'])],axis=1).drop('businesses',axis=1)
    return df